In [1]:
# Get raw data
with open('input/10.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [3]:
# Part 1
def get_buttons(x,y):
    buttons = np.zeros((len(x), y), dtype=int)
    buttons[*zip(*[[i,k] 
                   for i,j in enumerate(x) 
                   for k in eval(f'[{j[1:-1]}]')])] = 1
    return buttons

manual = [[(z:=np.array([[int(k=='#') for k in j[0][1:-1]]])), 
           get_buttons(j[1:-1], z.shape[1]), 
           np.array([int(k) for k in j[-1][1:-1].split(',')])]
          for i in rawinput.split('\n') 
          for j in [i.split()]]

hlist = lambda x: x.astype('<U1').view(f'<U{x.shape[1]}').reshape(-1).tolist()

tot_n_press = 0
for req, buttons, _ in manual:
    poss = np.zeros_like(req)
    hist = hlist(poss)
    n_press = 0
    while True:
        n_press += 1
        poss = np.logical_xor(np.repeat(poss, buttons.shape[0], axis=0),
                              np.tile(buttons, (poss.shape[0], 1))).astype(int)
        if np.any(np.all(poss == req, axis=1)):
            break
        poss = poss[((h:=np.array(hlist(poss)))[s:=np.lexsort((np.arange(h.shape[0]), h))] 
                     != np.append([-1], h[s][:-1]))
                    [np.argsort(s)]]
        poss = poss[~np.isin(hlist(poss), hist)]
        hist += hlist(poss)
    tot_n_press += n_press

tot_n_press

491

In [4]:
# Part 2
def divide_cols(work):
    pf = [2]
    while pf[-1] <= np.sqrt(np.max(work)):
        while ~np.all(work == (work:=np.where(np.all(np.mod(work, pf[-1]) == 0, 
                                                     axis=0, 
                                                     keepdims=True), 
                                              work//pf[-1], work))):
            pass
        pf += [pf[-1]+1]
        while any([pf[-1] % i == 0 for i in pf[:-1]]):  pf[-1] += 1
    return work

def get_search_values(x,y):
    sv = np.empty((1,0), dtype=int)
    for i in range(x):
        sv = np.column_stack((np.repeat(sv, y, axis=0), 
                              np.tile(np.arange(y), sv.shape[0])))
    return sv
    
def get_min_presses(r_buttons, r_req):
    work = np.append(r_buttons[np.argsort(-np.sum(r_buttons, axis=1))], [r_req], axis=0)
    
    r, c = 0, 0
    while all([i<j for i,j in zip([r,c], r_buttons.shape)]):
        if (y:=np.abs(work[r, c:]))[(z:=np.argmax(y))] == 0:
            r += 1
        else:
            work[:, [c, c+z]] = work[:, [c+z, c]]
            work *= (z:=np.where(np.mod(work[r:r+1], work[r:r+1, c]) != 0, 
                                 work[r:r+1, c], 1))
            work -= np.where(np.expand_dims(np.arange(work.shape[1]) == c, 0), 
                             0, 
                             work[r:r+1] // work[r:r+1, c] * work[:, c:c+1])
            work = divide_cols(work)
            r,c = r+1,c+1
            
    work = work[np.ix_(np.lexsort((np.arange(work.shape[0]), np.sum(work != 0, axis=1) > 1)),
                       np.arange(work.shape[1])[np.any(work != 0, axis=0)])]
    
    work *= np.sign(work[np.min(np.where(work==0, 
                                         work.shape[0]-1, 
                                         np.expand_dims(np.arange(work.shape[0]), 1)), 
                                axis=0), 
                         np.arange(work.shape[1])])
    
    if (nfree:=work.shape[0] - (ptr:=np.argmax(np.sum(work != 0, axis=1) > 1)) - 1):
        search_values = get_search_values(nfree, np.max(r_req))
        chsz = int(1e6)
        min_n_presses = None
        for i in range(0, search_values.shape[0], chsz):
            ch_res = np.sum(np.append((z:=(work[-1:] 
                                           - np.sum(np.expand_dims((svsub:=search_values[i:i+int(chsz)]), 2) 
                                                    * np.expand_dims(work[ptr:-1], 0), axis=1)) 
                                       / np.max(work[:ptr], axis=0)).astype(int), 
                                      svsub, axis=1)[np.all((z >= 0) & (np.mod(z, 1) == 0), axis=1)], axis=1)
            if ch_res.shape[0] and (min_n_presses is None or np.min(ch_res).item() < min_n_presses):
                min_n_presses = np.min(ch_res).item()
        return min_n_presses
    else:
        return np.sum(work[-1]).item()

sum([get_min_presses(buttons, req) for i, (_, buttons, req) in enumerate(manual)])

20617